__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dt
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
avito_path = 'sevastopol/kvartiry/prodam'


raw_data_path = 'data/raw/' # каталог с файлами объявлений
loc_file_path = 'data/location.pkl' # таблица адресов
result_file_path = 'data/data.pkl'

In [6]:
ts = dt.now().strftime('%Y-%m-%d_%H-%M')
ts

'2022-09-06_13-22'

## качаем новые данные

In [7]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoDownloaderRealty

df, html = AvitoDownloaderRealty(
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(avito_path, show_pbar=True)

print(len(df))

[INFO    ] 2022-09-06 13:09:56 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-06 13:09:56 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-06 13:09:58 | AvitoDownloader: downloader init
[INFO    ] 2022-09-06 13:10:06 | 80 pages for read
[INFO    ] 2022-09-06 13:10:06 | AvitoDownloader: start read and parse pages...


  0%|          | 0/79 [00:00<?, ?it/s]

[INFO    ] 2022-09-06 13:14:44 | DownloaderSeleniumFirefox: close virtual browser
3985
CPU times: user 17.2 s, sys: 712 ms, total: 18 s
Wall time: 4min 47s


In [8]:
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [9]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,avito_page
789,2340503980,"Объявление «1-к. квартира, 26 м², 1/3 эт.» 5 фотографий",3650954,ЖК «Голубая Бухта»,"аллея Защитников 35-й Батареи, д. 11а","ЖК «Голубая Бухта» — это неповторимое сочетание современной архитектуры и комфортной жизни с удобным расположением у самого берега Черного моря и живописным видом на Голубую бухту. \nКомплекс имеет уникальную инфраструктуру:300м до пляжа «Голубая бухта», Sp",16
1330,2512947430,"Объявление «1-к. квартира, 37,6 м², 1/10 эт.» 9 фотографий",6600000,,"ул. Тараса Шевченко, 24","Квартира расположена в новом районе. Рядом школа, детсад, рынок, магазины. До моря 25 мин пешком, 5 мин на машине. Свежий ремонт, этаж первый, высокий, в цокольном офисные помещения.",27


## собираем и чистим данные

In [7]:
import re
from os import listdir

raw_data_files = sorted([ raw_data_path+'/'+f for f in listdir(raw_data_path) if re.match(r'.+\.xlsx$',f) ])
raw_data_files

['data/raw//avito_2022-09-02_13-04_raw.xlsx',
 'data/raw//avito_2022-09-05_14-02_raw.xlsx',
 'data/raw//avito_2022-09-06_13-09_raw.xlsx']

In [8]:
from lib.avito import AvitoDataAggRealty

data = AvitoDataAggRealty().transform(raw_data_files)
print(len(data))

[INFO    ] 2022-09-06 13:22:43 | AvitoDataAggRealty: 3 raw data files
[INFO    ] 2022-09-06 13:22:44 | AvitoDataAggRealty: 11881 records
11881


In [9]:
assert len(data)>0

In [10]:
data.sample(2)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,nrooms,floor,...,area,is_studio,is_apartment,is_part,is_auction,is_openspace,is_roof,is_SNT,priceM,is_last_floor
424,2536748272,"1-к. квартира, 47,5 м², 5/10 эт.",7800000,NaN,"Античный проспект , 26/1","Объект №1947. \n\nПредлагаю к приобретению 1 комнатную квартиру в Жилом Комплексе «Жемчужина Севастополя»! \n\nЖилой комплекс привлекателен для современных и ведущих динамичный образ жизни людей, завораживающий вид на море и разнообразие инфраструктуры для Ваш",9,2022-09-02 13:04:00,1,5,...,47.50,False,False,False,False,False,False,False,7.80,False
11127,2484518108,"1-к. квартира, 32 м², 2/9 эт.",6000000,NaN,"проспект Героев Сталинграда, 46","Продается отличная 1ком квартира в одном из лучших районов города Севастополя. В квартире сделан ремонт, поменяны все коммуникации, окна, двери, сантехника, комната немного увеличена за счёт лоджии( не считается перепланировкой). \n\nВся инфраструктура в шаг",65,2022-09-06 13:09:00,1,2,...,32.00,False,False,False,False,False,False,False,6.00,False


### обновляем таблицу адресов

In [13]:
from lib.locator import LocationUpdater
from lib.locator import AddressTransformerSev

loc = LocationUpdater(
        address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-06 13:14:46 | LocationUpdater: 1672 addresses in location table
[INFO    ] 2022-09-06 13:14:46 | LocationUpdater: 1746 addresses total
[INFO    ] 2022-09-06 13:14:46 | LocationUpdater: 1672 addresses defined
[INFO    ] 2022-09-06 13:14:46 | LocationUpdater: 74 addresses undefined


  0%|          | 0/74 [00:00<?, ?it/s]

[INFO    ] 2022-09-06 13:15:58 | LocationUpdater: 34 new addresses found


In [14]:
loc.sample(3)

,adr,latitude,longitude,truncated
900,"улица Хрусталёва, 149",44.56,33.52,False
89,"улица Льва Толстого, 6",44.68,33.73,False
76,"Казачья улица , 23",44.57,33.42,False


In [15]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [16]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 11881
записей без геометки: 637


In [17]:
assert len(data)>0
data.to_pickle(result_file_path)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11881 entries, 0 to 11880
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   avito_id       11881 non-null  int64         
 1   title          11881 non-null  object        
 2   price          11881 non-null  int64         
 3   obj_name       2958 non-null   object        
 4   adr            11881 non-null  object        
 5   description    11869 non-null  object        
 6   avito_page     11881 non-null  int64         
 7   ts             11881 non-null  datetime64[ns]
 8   nrooms         11881 non-null  int64         
 9   floor          11881 non-null  int64         
 10  nfloors        11881 non-null  int64         
 11  area           11881 non-null  float64       
 12  is_studio      11881 non-null  bool          
 13  is_apartment   11881 non-null  bool          
 14  is_part        11881 non-null  bool          
 15  is_auction     1188